In [38]:
!pip install fastparquet

   ---------------------------------------- 0.0/673.3 kB ? eta -:--:--
   - ------------------------------------- 30.7/673.3 kB 640.0 kB/s eta 0:00:02
   ------- -------------------------------- 122.9/673.3 kB 1.2 MB/s eta 0:00:01
   ----------------------------- ---------- 501.8/673.3 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------  665.6/673.3 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 673.3/673.3 kB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------------- -------------------------- 0.7/2.1 MB 23.1 MB/s eta 0:00:01
   ---------------------------- ----------- 1.5/2.1 MB 23.9 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 22.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 16.7 MB/s eta 0:00:00


In [40]:
import kagglehub
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset

ImportError: The pyarrow installation is not built with support for the Parquet file format (DLL load failed while importing _parquet: The specified procedure could not be found.)

**Data Prep**

In [13]:
# Download the dataset to the data directory
path = kagglehub.dataset_download("sunilthite/llm-detect-ai-generated-text-dataset")

100%|██████████| 18.6M/18.6M [00:01<00:00, 17.4MB/s]

Extracting files...


In [15]:
df = pd.read_csv(os.path.join(path, "Training_Essay_Data.csv"))
df.head()

,text,generated
0,Car-free cities have become a subject of incre...,1
1,"Car Free Cities Car-free cities, a concept ga...",1
2,A Sustainable Urban Future Car-free cities ...,1
3,Pioneering Sustainable Urban Living In an e...,1
4,The Path to Sustainable Urban Living In an ...,1


In [18]:
df = df.rename(columns={'generated': 'label'})
print(f"Dataset shape: {df.shape}")
print(f"Column names: {df.columns.tolist()}")
print(f"Label distribution: {df['label'].value_counts()}")

Dataset shape: (29145, 2)
Column names: ['text', 'label']
Label distribution: label
0    17508
1    11637
Name: count, dtype: int64


In [20]:
# Split the data into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

In [22]:
print(f"Train set: {train_df.shape[0]} samples")
print(f"Validation set: {val_df.shape[0]} samples")
print(f"Test set: {test_df.shape[0]} samples")

Train set: 20401 samples
Validation set: 4372 samples
Test set: 4372 samples


In [24]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

NameError: name 'Dataset' is not defined